Version 1.0.3

# Pandas basics 

Hi! In this programming assignment you need to refresh your `pandas` knowledge. You will need to do several [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)s and [`join`]()`s to solve the task. 

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

from grader import Grader

In [56]:
DATA_FOLDER = '../readonly/final_project_data/'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

The dataset we are going to use is taken from the competition, that serves as the final project for this course. You can find complete data description at the [competition web page](https://www.kaggle.com/c/competitive-data-science-final-project/data). To join the competition use [this link](https://www.kaggle.com/t/1ea93815dca248e99221df42ebde3540).

## Grading

We will create a grader instace below and use it to collect your answers. When function `submit_tag` is called, grader will store your answer *locally*. The answers will *not* be submited to the platform immediately so you can call `submit_tag` function as many times as you need. 

When you are ready to push your answers to the platform you should fill your credentials and run `submit` function in the <a href="#Authorization-&-Submission">last paragraph</a>  of the assignment.

In [4]:
grader = Grader()

# Task

Let's start with a simple task. 

<ol start="0">
  <li><b>Print the shape of the loaded dataframes and use [`df.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function to print several rows. Examine the features you are given.</b></li>
</ol>

In [57]:
# YOUR CODE GOES HERE
transactions = transactions.dropna()
print(transactions.shape)
transactions.head(10)


(2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


Now use your `pandas` skills to get answers for the following questions. 
The first question is:

1. ** What was the maximum total revenue among all the shops in September, 2014?** 


* Hereinafter *revenue* refers to total sales minus value of goods returned.

*Hints:*

* Sometimes items are returned, find such examples in the dataset. 
* It is handy to split `date` field into [`day`, `month`, `year`] components and use `df.year == 14` and `df.month == 9` in order to select target subset of dates.
* You may work with `date` feature as with strings, or you may first convert it to `pd.datetime` type with `pd.to_datetime` function, but do not forget to set correct `format` argument.

In [58]:
transactions.date = pd.to_datetime(transactions.date, format='%d.%m.%Y')
transactions['year'] = transactions.date.dt.year
transactions['month'] = transactions.date.dt.month

In [59]:
cond1 = (transactions.year == 2014) & (transactions.month == 9)
transactions['day_trnsct_value'] = transactions.item_cnt_day * transactions.item_price
print('Revenues in desc order')
print(transactions[cond1].groupby('shop_id')['day_trnsct_value'].sum().sort_values(ascending=False))

Revenues in desc order
shop_id
31    7.982852e+06
25    6.783338e+06
12    6.378335e+06
28    4.985847e+06
27    4.899292e+06
54    4.839335e+06
42    4.753366e+06
6     3.480397e+06
57    3.223198e+06
43    2.793483e+06
46    2.703858e+06
26    2.504839e+06
47    2.487424e+06
38    2.472334e+06
15    2.383940e+06
35    2.323137e+06
22    2.175790e+06
18    2.132611e+06
7     2.127491e+06
29    2.071339e+06
19    2.050485e+06
21    2.023241e+06
24    1.956968e+06
58    1.949760e+06
16    1.919162e+06
53    1.914577e+06
30    1.902539e+06
50    1.872948e+06
55    1.794684e+06
56    1.780582e+06
59    1.567666e+06
52    1.520397e+06
14    1.499621e+06
2     1.473540e+06
48    1.463744e+06
37    1.401708e+06
17    1.385955e+06
5     1.350828e+06
3     1.228425e+06
4     1.211556e+06
45    1.199660e+06
41    1.133782e+06
44    1.049887e+06
49    8.754520e+05
39    7.788630e+05
10    7.744040e+05
51    7.650010e+05
33    5.391150e+05
34    4.829700e+05
40    3.538670e+05
Name: day_trnsct_va

In [60]:
# YOUR CODE GOES HERE

max_revenue = transactions[cond1].groupby('shop_id')['day_trnsct_value'].sum().sort_values(ascending=False).iloc[0] # PUT YOUR ANSWER IN THIS VARIABLE
grader.submit_tag('max_revenue', max_revenue)

Current answer for task max_revenue is: 7982852.2


Great! Let's move on and answer another question:

<ol start="2">
  <li><b>What item category generated the highest revenue in summer 2014?</b></li>
</ol>

* Submit `id` of the category found.
    
* Here we call "summer" the period from June to August.

*Hints:*

* Note, that for an object `x` of type `pd.Series`: `x.argmax()` returns **index** of the maximum element. `pd.Series` can have non-trivial index (not `[1, 2, 3, ... ]`).

In [61]:
cond1 = (transactions.year == 2014) & (transactions.month >= 6) & (transactions.month <= 8)
transactions = transactions.join(items, on='item_id', lsuffix='_trns', rsuffix='_items')
transactions['day_trnsct_value'] = transactions.item_cnt_day * transactions.item_price
print('Top revenue, Corresponding Category_id')

print(transactions[cond1].groupby('item_category_id')['day_trnsct_value'].sum().sort_values(ascending=False).iloc[0],
      transactions[cond1].groupby('item_category_id')['day_trnsct_value'].sum().argmax())

Top revenue, Corresponding Category_id
32157302.43 20


In [9]:
transactions.head()

,date,date_block_num,shop_id,item_id_trns,item_price,item_cnt_day,year,month,day_trnsct_value,item_name,item_id_items,item_category_id
0,2013-01-02,0,59,22154,999.00,1.0,2013,1,999.00,ЯВЛЕНИЕ 2012 (BD),22154,37
1,2013-01-03,0,25,2552,899.00,1.0,2013,1,899.00,DEEP PURPLE The House Of Blue Light LP,2552,58
2,2013-01-05,0,25,2552,899.00,-1.0,2013,1,-899.00,DEEP PURPLE The House Of Blue Light LP,2552,58
3,2013-01-06,0,25,2554,1709.05,1.0,2013,1,1709.05,DEEP PURPLE Who Do You Think We Are LP,2554,58
4,2013-01-15,0,25,2555,1099.00,1.0,2013,1,1099.00,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),2555,56


In [10]:
# YOUR CODE GOES HERE

category_id_with_max_revenue = 20 # PUT YOUR ANSWER IN THIS VARIABLE
grader.submit_tag('category_id_with_max_revenue', category_id_with_max_revenue)

Current answer for task category_id_with_max_revenue is: 20


<ol start="3">
  <li><b>How many items are there, such that their price stays constant (to the best of our knowledge) during the whole period of time?</b></li>
</ol>

* Let's assume, that the items are returned for the same price as they had been sold.

In [62]:
trns1 = transactions.groupby('item_id_trns')['item_price'].nunique()
trns1[trns1 == 1].sum()

5926

In [51]:
# YOUR CODE GOES HERE

num_items_constant_price = trns1[trns1 == 1].sum()# PUT YOUR ANSWER IN THIS VARIABLE
grader.submit_tag('num_items_constant_price', num_items_constant_price)

Current answer for task num_items_constant_price is: 5926


Remember, the data can sometimes be noisy.

<ol start="4">
  <li><b>What was the variance of the number of sold items per day sequence for the shop with `shop_id = 25` in December, 2014? Do not count the items, that were sold but returned back later.</b></li>
</ol>

* Fill `total_num_items_sold` and `days` arrays, and plot the sequence with the code below.
* Then compute variance. Remember, there can be differences in how you normalize variance (biased or unbiased estimate, see [link](https://math.stackexchange.com/questions/496627/the-difference-between-unbiased-biased-estimator-variance)). Compute ***unbiased*** estimate (use the right value for `ddof` argument in `pd.var` or `np.var`). 
* If there were no sales at a given day, ***do not*** impute missing value with zero, just ignore that day

In [102]:
cond = (transactions.shop_id == 25) & (transactions.year == 2014) & (transactions.month == 12)
trns2 = transactions[cond].groupby('date')['item_cnt_day'].sum()
trns2[trns2 != 0].var()

117167.70229885059

In [103]:
total_num_items_sold_var = trns2[trns2 != 0].var() # PUT YOUR ANSWER IN THIS VARIABLE
grader.submit_tag('total_num_items_sold_var', total_num_items_sold_var)

Current answer for task total_num_items_sold_var is: 117167.702299


In [97]:
shop_id = 25

total_num_items_sold = trns2[trns2 >= 0].sum()
days = # 

# Plot it
plt.plot(days, total_num_items_sold)
plt.ylabel('Num items')
plt.xlabel('Day')
plt.title("Daily revenue for shop_id = 25")
plt.show()

total_num_items_sold_var = # PUT YOUR ANSWER IN THIS VARIABLE
grader.submit_tag('total_num_items_sold_var', total_num_items_sold_var)

SyntaxError: invalid syntax (<ipython-input-97-afc26729a42d>, line 4)

## Authorization & Submission
To submit assignment to Cousera platform, please, enter your e-mail and token into the variables below. You can generate token on the programming assignment page. *Note:* Token expires 30 minutes after generation.

In [104]:
STUDENT_EMAIL = 'jere0036@ntu.edu.sg'
STUDENT_TOKEN = 'Mx08b2ZJggcqmpXJ' # TOKEN HERE
grader.status()

You want to submit these numbers:
Task max_revenue: 7982852.2
Task category_id_with_max_revenue: 20
Task num_items_constant_price: 5926
Task total_num_items_sold_var: 117167.702299


In [105]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!


Well done! :)